In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, clear_output

In [2]:
import kagglehub
path = kagglehub.dataset_download("tanishksharma9905/global-economic-indicators-20102025")
print("Path to dataset files:", path)

100%|██████████| 257k/257k [00:00<00:00, 68.7MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/tanishksharma9905/global-economic-indicators-20102025/versions/1


In [3]:
df = pd.read_csv("/root/.cache/kagglehub/datasets/tanishksharma9905/global-economic-indicators-20102025/versions/1/world_bank_data_2025.csv")
df.head()

,country_name,country_id,year,Inflation (CPI %),GDP (Current USD),GDP per Capita (Current USD),Unemployment Rate (%),"Interest Rate (Real, %)","Inflation (GDP Deflator, %)",GDP Growth (% Annual),Current Account Balance (% GDP),Government Expense (% of GDP),Government Revenue (% of GDP),Tax Revenue (% of GDP),Gross National Income (USD),Public Debt (% of GDP)
0,Aruba,aw,2010,2.078141,2.453597e+09,24093.140151,NaN,11.666131,-1.223407,-2.733457,-18.752537,NaN,NaN,NaN,2.313385e+09,NaN
1,Aruba,aw,2011,4.316297,2.637859e+09,25712.384302,NaN,4.801974,4.005674,3.369237,-9.877656,NaN,NaN,NaN,2.391841e+09,NaN
2,Aruba,aw,2012,0.627472,2.615208e+09,25119.665545,NaN,8.200875,0.184033,-1.040800,3.473451,NaN,NaN,NaN,2.499118e+09,NaN
3,Aruba,aw,2013,-2.372065,2.727850e+09,25813.576727,NaN,10.709709,-1.995948,6.431483,-11.813206,NaN,NaN,NaN,2.563517e+09,NaN
4,Aruba,aw,2014,0.421441,2.790850e+09,26129.839062,NaN,3.213869,3.958897,-1.586575,-4.658577,NaN,NaN,NaN,2.688102e+09,NaN


In [5]:
df = df.drop(columns=['country_id'], errors='ignore')
df.fillna(0, inplace=True)
df.head()

,country_name,year,Inflation (CPI %),GDP (Current USD),GDP per Capita (Current USD),Unemployment Rate (%),"Interest Rate (Real, %)","Inflation (GDP Deflator, %)",GDP Growth (% Annual),Current Account Balance (% GDP),Government Expense (% of GDP),Government Revenue (% of GDP),Tax Revenue (% of GDP),Gross National Income (USD),Public Debt (% of GDP)
0,Aruba,2010,2.078141,2.453597e+09,24093.140151,0.0,11.666131,-1.223407,-2.733457,-18.752537,0.0,0.0,0.0,2.313385e+09,0.0
1,Aruba,2011,4.316297,2.637859e+09,25712.384302,0.0,4.801974,4.005674,3.369237,-9.877656,0.0,0.0,0.0,2.391841e+09,0.0
2,Aruba,2012,0.627472,2.615208e+09,25119.665545,0.0,8.200875,0.184033,-1.040800,3.473451,0.0,0.0,0.0,2.499118e+09,0.0
3,Aruba,2013,-2.372065,2.727850e+09,25813.576727,0.0,10.709709,-1.995948,6.431483,-11.813206,0.0,0.0,0.0,2.563517e+09,0.0
4,Aruba,2014,0.421441,2.790850e+09,26129.839062,0.0,3.213869,3.958897,-1.586575,-4.658577,0.0,0.0,0.0,2.688102e+09,0.0


In [16]:
numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
numeric_cols_with_none = ['None'] + numeric_cols

In [7]:
country_dropdown = widgets.Dropdown(
    options=sorted(df['country_name'].unique()),
    description='Country:',
    value='India'
)

In [18]:
x_indicator_dropdown = widgets.Dropdown(
    options=numeric_cols_with_none,
    description='X Axis:',
    value='GDP Growth (% Annual)'
)

In [17]:
y_indicator_dropdown = widgets.Dropdown(
    options=numeric_cols_with_none,
    description='Y Axis:',
    value='Inflation (CPI %)'
)

In [9]:
year_slider = widgets.IntRangeSlider(
    value=[2010, 2025],
    min=int(df['year'].min()),
    max=int(df['year'].max()),
    step=1,
    description='Years:',
    continuous_update=False
)

In [11]:
plot_type_toggle = widgets.ToggleButtons(
    options=['Line Plot', 'Heatmap', 'Scatter'],
    description='Plot:',
    value='Line Plot'
)

In [12]:
output = widgets.Output()

In [19]:
def update_dashboard(change=None):
    with output:
        clear_output(wait=True)

        country = country_dropdown.value
        x_ind = x_indicator_dropdown.value
        y_ind = y_indicator_dropdown.value
        year_start, year_end = year_slider.value
        plot_type = plot_type_toggle.value

        subset = df[
            (df['country_name'] == country) &
            (df['year'] >= year_start) &
            (df['year'] <= year_end)
        ]

        if subset.empty:
            print("No data available for selection.")
            return

        if plot_type == 'Line Plot':
            plt.figure(figsize=(14, 6))
            plotted = False
            if x_ind != 'None':
                sns.lineplot(data=subset, x='year', y=x_ind, label=x_ind)
                plotted = True
            if y_ind != 'None':
                sns.lineplot(data=subset, x='year', y=y_ind, label=y_ind)
                plotted = True
            if plotted:
                plt.title(f"{country}: Line Plot ({year_start}–{year_end})")
                plt.xlabel("Year")
                plt.ylabel("Value")
                plt.grid(True)
                plt.legend()
                plt.tight_layout()
                plt.show()
            else:
                print("Please select at least one indicator for Line Plot.")

        elif plot_type == 'Heatmap':
            plt.figure(figsize=(8, 6))
            corr = subset[numeric_cols].corr()
            sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")
            plt.title(f"{country}: Correlation Heatmap ({year_start}–{year_end})")
            plt.tight_layout()
            plt.show()

        elif plot_type == 'Scatter':
            if x_ind != 'None' and y_ind != 'None':
                plt.figure(figsize=(8, 6))
                sns.scatterplot(data=subset, x=x_ind, y=y_ind, hue='year', palette='viridis', s=100)
                plt.title(f"{country}: {y_ind} vs. {x_ind} ({year_start}–{year_end})")
                plt.grid(True)
                plt.tight_layout()
                plt.show()
            else:
                print("Please select both X and Y indicators for Scatter Plot.")

In [20]:
country_dropdown.observe(update_dashboard, names='value')
x_indicator_dropdown.observe(update_dashboard, names='value')
y_indicator_dropdown.observe(update_dashboard, names='value')
year_slider.observe(update_dashboard, names='value')
plot_type_toggle.observe(update_dashboard, names='value')

In [21]:
dashboard_ui = widgets.VBox([
    widgets.HBox([country_dropdown, plot_type_toggle]),
    widgets.HBox([x_indicator_dropdown, y_indicator_dropdown]),
    year_slider,
    output
])

display(dashboard_ui)
update_dashboard()